In [1]:
import numpy as np
import pandas as pd
from math import *
from numpy.linalg import inv,det

In [2]:
constraints=pd.read_excel('constraints.xlsx')[1:]
constraints

,H_i,Unnamed: 1,B_j,Unnamed: 3,Unnamed: 4,B_i,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
1,1,aphid,NaN,NaN,NaN,0,NaN,NaN,t*pi,pi,0
2,2,ax,NaN,NaN,NaN,0,0 0,NaN,0,NaN,NaN
3,3,ay,NaN,NaN,NaN,0,0 0,NaN,0,NaN,NaN
4,4,ax,NaN,NaN,NaN,1,0 0,NaN,0,NaN,NaN
5,5,ay,NaN,NaN,NaN,1,0 0,NaN,-2,NaN,NaN
6,6,rt,0,2 0,NaN,1,0 0,1 0,0,NaN,NaN


In [43]:
# for index, row in constraints.iterrows():
#     print(index)
#     print()
#     print(row)
#     print()

In [31]:
def gamma_ax(q,dq,row):
    i=row['B_i']
    phii=q[i*3+2,0]
    dphii=dq[i*3+2,0]
    Ai=np.mat([[m.cos(phii),-m.sin(phii)],[m.sin(phii),m.cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    x=np.mat([1,0]).T

    return x.T*Ai*si*dphii*dphii

def gamma_ay(q,dq,row):
    i=row['B_i']
    phii=q[i*3+2,0]
    dphii=dq[i*3+2,0]
    Ai=np.mat([[m.cos(phii),-m.sin(phii)],[m.sin(phii),m.cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    y=np.mat([0,1]).T

    return y.T*Ai*si*dphii*dphii

def gamma_rt(q,dq,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    
    j=row['B_j']
    xj=q[j*3,0]
    yj=q[j*3+1,0]
    phij=q[j*3+2,0]
    rj=np.mat([xj,yj]).T
    Aj=np.mat([[cos(phij),-sin(phij)],[sin(phij),cos(phij)]])
    numbers = row['Unnamed: 3'].split(' ')
    numbers = [ float(x) for x in numbers ]
    sj=np.mat(numbers).T

    numbers = row['Unnamed: 7'].split(' ')
    numbers = [ float(x) for x in numbers ]
    v=np.mat(numbers).T
    
    c=float(row['Unnamed: 8'])
    R=np.mat([[0,-1],[1,0]])

    h=rj+Aj*sj-ri-Ai*si

    item1=(-2)
    return (R*Ai*v).T*h/np.linalg.norm(v)-c

In [40]:
def calculate_Gamma(q, dq, t, constraints):
    Gamma = np.mat([0])
    for index, row in constraints.iterrows():
#         if row['Unnamed: 1'] == 'r':
#             thisGamma = gamma_r(q,dq,row)
            
#         if row['Unnamed: 1']=='aphid':
#             thisGamma = gamma_aphid(q,t,row)
        if row['Unnamed: 1']=='ax':
            thisGamma = gamma_ax(q,dq,row)
            Gamma=np.vstack((Gamma,thisGamma))
        if row['Unnamed: 1']=='ay':
            thisGamma = gamma_ay(q,dq,row)
            Gamma=np.vstack((Gamma,thisGamma))

    return Gamma[1:]
t=1
q=np.mat([0,0,0]).T
dq=np.mat([0,0,1]).T
calculate_Gamma(q, dq, t, constraints)

matrix([[-0.5],
        [ 0. ]])

In [41]:
def v_r(q,row):
    return np.mat([0,0]).T
def v_ax(q,row):
    return np.mat([0]).T
def v_ay(q,row):
    return np.mat([0]).T
def v_aphi(q,row):
    return np.mat([0]).T

In [44]:
def calculate_V(q, t, constraints):
    V = np.mat([0])
    for index, row in constraints.iterrows():
        if row['Unnamed: 1'] == 'ax':
            thisV = v_ax(q,row)
            V=np.vstack((V,thisV))
#         if row['Unnamed: 1']=='aphid':
#             thisV = v_aphid(q,t,row)
#             V=np.vstack((V,thisV))
        if row['Unnamed: 1'] == 'ay':
            thisV = v_ay(q,row)
            V=np.vstack((V,thisV))
    return V[1:]
t=1
q=np.mat([0,0,0]).T
dq=np.mat([0,0,1]).T
calculate_V(q, t, constraints)

matrix([[0],
        [0]])

In [53]:
def phiq_ax(q,row,n):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[m.cos(phii),-m.sin(phii)],[m.sin(phii),m.cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    
    x=np.mat([1,0]).T
    R=np.mat([[0,-1],[1,0]])
    
    thisPHIq = x.T*R*Ai*si
    thisPHIq = np.hstack((x.T,thisPHIq))
    
    if i>0:
        thisPHIql = np.mat([0]*i*6).reshape(2,-1)
        thisPHIq = np.hstack((thisPHIql,thisPHIq))

    if i*3+3<n :
        thisPHIqr = np.mat([0]*(n-i*3-3)).reshape(1,-1)
        thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    return thisPHIq

def phiq_ay(q,row,n):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[m.cos(phii),-m.sin(phii)],[m.sin(phii),m.cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    
    y=np.mat([0,1]).T
    R=np.mat([[0,-1],[1,0]])
    
    thisPHIq = y.T*R*Ai*si
    thisPHIq = np.hstack((y.T,thisPHIq))
    
    if i>0:
        thisPHIql = np.mat([0]*i*6).reshape(2,-1)
        thisPHIq = np.hstack((thisPHIql,thisPHIq))

    if i*3+3<n :
        thisPHIqr = np.mat([0]*(n-i*3-3)).reshape(1,-1)
        thisPHIq = np.hstack((thisPHIq,thisPHIqr))
    return thisPHIq

In [56]:
def calculate_PHIq(q, t, constraints):
    PHIq = np.mat([0]*q.shape[0])

    di=constraints['B_i']
    dj=constraints['B_j']
    df=pd.concat([di,dj], ignore_index=True)
    n = df.value_counts().shape[0]*3

    for index, row in constraints.iterrows():
        if row['Unnamed: 1'] == 'ax':
            thisPHIq = phiq_ax(q,row,n)
            PHIq=np.vstack((PHIq,thisPHIq))

        if row['Unnamed: 1']=='ay':
            thisPHIq = phiq_ay(q,row,n)

            PHIq=np.vstack((PHIq,thisPHIq))

    return PHIq[1:]
t=1
q=np.mat([0,0,m.pi]).T

calculate_PHIq(q, t, constraints)

matrix([[1.000000e+00, 0.000000e+00, 6.123234e-17],
        [0.000000e+00, 1.000000e+00, 5.000000e-01]])

In [44]:
def phi_ax(q,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    x=np.mat([1,0]).T
    return x.T*(ri+Ai*si)-ci

def phi_ay(q,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    ci=float(row['Unnamed: 8'])
    y=np.mat([0,1]).T
    return y.T*(ri+Ai*si)-ci

def phi_rt(q,row):
    i=row['B_i']
    xi=q[i*3,0]
    yi=q[i*3+1,0]
    phii=q[i*3+2,0]
    ri=np.mat([xi,yi]).T
    Ai=np.mat([[cos(phii),-sin(phii)],[sin(phii),cos(phii)]])
    numbers = row['Unnamed: 6'].split(' ')
    numbers = [ float(x) for x in numbers ]
    si=np.mat(numbers).T
    
    j=row['B_j']
    xj=q[j*3,0]
    yj=q[j*3+1,0]
    phij=q[j*3+2,0]
    rj=np.mat([xj,yj]).T
    Aj=np.mat([[cos(phij),-sin(phij)],[sin(phij),cos(phij)]])
    numbers = row['Unnamed: 3'].split(' ')
    numbers = [ float(x) for x in numbers ]
    sj=np.mat(numbers).T

    numbers = row['Unnamed: 7'].split(' ')
    numbers = [ float(x) for x in numbers ]
    v=np.mat(numbers).T
    
    c=float(row['Unnamed: 8'])
    R=np.mat([[0,-1],[1,0]])

    h=rj+Aj*sj-ri-Ai*si
    
    return (R*Ai*v).T*h/np.linalg.norm(v)-c

In [47]:
def calculate_PHI(q, t, constraints):
    PHI = np.mat([0])
    for index, row in constraints.iterrows():
#         if row['Unnamed: 1'] == 'r':
#             thisPHI = phi_r(q,row)
#         if row['Unnamed: 1']=='ay':
#             thisPHI = phi_ay(q,row)
#             PHI=np.vstack((PHI,thisPHI))
#         if row['Unnamed: 1'] == 'ax':
#             thisPHI = phi_ax(q,row)
#             PHI=np.vstack((PHI,thisPHI))
        if row['Unnamed: 1'] == 'rt':
            thisPHI = phi_rt(q,row)
            PHI=np.vstack((PHI,thisPHI))
    return PHI[1:]
t=1
q=np.mat([0,0,pi/6,0,-2,pi/3]).T
calculate_PHI(q, t, constraints)

matrix([[4.4408921e-16]])